In [6]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
#목표 : 언론사 홈페이지에서 오늘 가장 이슈가 되는 기사 뽑기
url = "https://news.naver.com/main/ranking/popularDay.naver"
header = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
res = requests.get(url,headers = header)
res.text

'\n\n\n\n\n\n\n\n<!DOCTYPE HTML>\n<html lang="ko">\n<head>\n    <meta charset="euc-kr">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="referrer" contents="always">\n    <meta http-equiv="refresh" content="600"/>\n    <meta name="viewport" content="width=1106"/>\n    <meta name="format-detection" content="telephone=no"/>\n    \r\n    \r\n    \r\n        \r\n            \r\n            \r\n            \r\n            \r\n            \r\n        \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n\r\n<meta property="og:title"       content="네이버 뉴스">\r\n<meta property="og:type"        content="website">\r\n<meta property="og:url"         content="https://news.naver.com/main/home.naver">\r\n<meta property="og:image"       content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png"/>\r\n<meta property="og:description" content="정치, 경제, 사회, 생활/문

In [7]:
soup = BeautifulSoup(res.text)


In [10]:
all_box = soup.find_all("div", attrs = {"class" : "rankingnews_box"})
#가져올 데이터 : 날짜정보, 신문사 이름, 뉴스타이틀
#len(all_box)


82

In [15]:
lst_all_data = []
for box in all_box:
    company_name =  box.strong.text
    lst_all_rank = box.find_all("li")
    for rank in lst_all_rank:
        lst_data = []
        lst_data.append(company_name)#신문사 이름 들어가는 리스트
        #data가 없는 것들 처리
        if (rank.a == None):
            continue
        
        lst_data.append(rank.a.text) #기사 제목 정보를 리스트에 넣기
        lst_data.append(rank.a["href"]) #기사 url 정보를 리스트에 넣기
        lst_all_data.append(lst_data)

#dataFrame으로 만들기
df = pd.DataFrame(lst_all_data,  columns= ["company" , "title",  "link"])
df.to_csv("NaverNews.csv", index = False) #인덱스 빼고 저장

In [16]:
df = pd.read_csv("NaverNews.csv")
df["title"] = df["title"].str.replace("[^가-힣]", "" , regex = True) #한글을 제외한 문자를 모두 없애는 정규표현식
df

,company,title,link
0,연합뉴스,일째주차장막은차량경찰체포압수영장신청한다,https://n.news.naver.com/article/001/001402815...
1,연합뉴스,침대선구자에이스침대창업주안유수회장별세향년세종합,https://n.news.naver.com/article/001/001402813...
2,연합뉴스,월모평국어만점자작년수능의배정작수학이불수학,https://n.news.naver.com/article/001/001402810...
3,연합뉴스,보농심신라면원새우깡원인하,https://n.news.naver.com/article/001/001402845...
4,연합뉴스,화이트리스트년만에복원양국신뢰완전회복종합보,https://n.news.naver.com/article/001/001402829...
...,...,...,...
400,레이디경향,심장건강에특히좋은견과류이것,https://n.news.naver.com/article/145/000001904...
401,레이디경향,쿰쿰한장마철욕실전문가의청소법은요,https://n.news.naver.com/article/145/000001904...
402,레이디경향,젤네일말리는램프손상시킨다,https://n.news.naver.com/article/145/000001905...
403,레이디경향,이제사무실에서도블루보틀분말형태인스턴트커피출시,https://n.news.naver.com/article/145/000001904...


In [19]:
#의미있는 단어 뽑기 (형태소 분석 pip install kiwipiepy 패키지 이용)
from kiwipiepy import Kiwi
kiwi = Kiwi() #인스턴스 만들어서 사용해야함

#kiwi.tokenize("이제사무실에서도블루보틀분말형태인스턴트커피출시	") #tag = 8품사 정보 나타냄

df["형태소분석결과"] = df["title"].apply(lambda x : kiwi.tokenize(x)) #새로 만든 column


,company,title,link,형태소분석결과
0,연합뉴스,일째주차장막은차량경찰체포압수영장신청한다,https://n.news.naver.com/article/001/001402815...,"[(일, NNG, 0, 1), (째, XSN, 1, 1), (주차장, NNG, 2,..."
1,연합뉴스,침대선구자에이스침대창업주안유수회장별세향년세종합,https://n.news.naver.com/article/001/001402813...,"[(침대, NNG, 0, 2), (선구자, NNG, 2, 3), (에이스침대, NN..."
2,연합뉴스,월모평국어만점자작년수능의배정작수학이불수학,https://n.news.naver.com/article/001/001402810...,"[(월, NNG, 0, 1), (모평, NNG, 1, 2), (국어, NNG, 3,..."
3,연합뉴스,보농심신라면원새우깡원인하,https://n.news.naver.com/article/001/001402845...,"[(보, NNB, 0, 1), (농심, NNP, 1, 2), (신라면, NNG, 3..."
4,연합뉴스,화이트리스트년만에복원양국신뢰완전회복종합보,https://n.news.naver.com/article/001/001402829...,"[(화이트, NNP, 0, 3), (리스트, NNG, 3, 3), (년, NNB, ..."
...,...,...,...,...
400,레이디경향,심장건강에특히좋은견과류이것,https://n.news.naver.com/article/145/000001904...,"[(심장, NNG, 0, 2), (건강, NNG, 2, 2), (에, JKB, 4,..."
401,레이디경향,쿰쿰한장마철욕실전문가의청소법은요,https://n.news.naver.com/article/145/000001904...,"[(쿰쿰, NNG, 0, 2), (하, XSA, 2, 1), (ᆫ, ETM, 2, ..."
402,레이디경향,젤네일말리는램프손상시킨다,https://n.news.naver.com/article/145/000001905...,"[(젤, MAG, 0, 1), (네일, NNG, 1, 2), (말리, VV, 3, ..."
403,레이디경향,이제사무실에서도블루보틀분말형태인스턴트커피출시,https://n.news.naver.com/article/145/000001904...,"[(이제, MAG, 0, 2), (사무실, NNG, 2, 3), (에서, JKB, ..."


In [22]:
df_word = df["형태소분석결과"].explode().reset_index(drop = True) # df["형태소분석결과"] column 쭉 펼친 결과 (explode()함수 활용)
df_word 
#drop = True : 기존의 인덱스 없애기

0         (일, NNG, 0, 1)
1         (째, XSN, 1, 1)
2       (주차장, NNG, 2, 3)
3          (막, VV, 5, 1)
4         (은, ETM, 6, 1)
              ...       
5164      (에, JKB, 9, 1)
5165       (ᆫ, JX, 9, 1)
5166    (강렬하, VA, 10, 3)
5167      (게, EF, 13, 1)
5168    (화보, NNG, 14, 2)
Name: 형태소분석결과, Length: 5169, dtype: object

In [24]:
#타겟이 되는 단어 고르는 기준
target = ["NNG" , "NNP" , "VV",  "VA" , "XR" , "SL"]

word = [] #타겟 단어만 담는 리스트
for token in df_word:
    if (type(token) == float): #여기부분 이해 안감!!!!!!
        continue
    if token[1] in target:
        word.append(token[0]) #단어 부분 추가
word


['일',
 '주차장',
 '막',
 '차량',
 '경찰',
 '체포',
 '압수',
 '영장',
 '신청',
 '침대',
 '선구자',
 '에이스침대',
 '창업주',
 '안',
 '유수',
 '회장',
 '별세',
 '향년',
 '종합',
 '월',
 '모평',
 '국어',
 '만점자',
 '작년',
 '수능',
 '배',
 '수학',
 '이불',
 '수학',
 '농심',
 '신라면',
 '새우깡',
 '인하',
 '화이트',
 '리스트',
 '복원',
 '양국',
 '신뢰',
 '완전',
 '회복',
 '종합',
 '보',
 '거제',
 '거북선',
 '최후',
 '목재',
 '소각',
 '금속',
 '고물상',
 '과태료',
 '내',
 '인도',
 '주차',
 '반복',
 '신고',
 '가능',
 '앵커',
 '리포트',
 '양육비',
 '내',
 '아이',
 '아빠',
 '얼굴',
 '공개',
 '시위',
 '미혼모',
 '유죄',
 '날씨',
 '내륙',
 '장마',
 '소강',
 '무더위',
 '기승',
 '제주도',
 '남해안',
 '장맛비',
 '프리',
 '고진',
 '정권',
 '전복',
 '불',
 '푸틴',
 '반란',
 '실패',
 '운명',
 '진통제',
 '맞',
 '얼음',
 '찜',
 '김영철',
 '지나치',
 '운동',
 '걸리',
 '병',
 '건강',
 '자',
 '일어나',
 '일',
 '황희찬',
 '가짜',
 '메시지',
 '경고',
 '하루',
 '주차',
 '만원',
 '나오',
 '인천',
 '오피스텔',
 '서',
 '일',
 '유상증자',
 '단위',
 '하',
 '대기업',
 '개미',
 '쇼크',
 '네이버',
 '카카오',
 '주가',
 '추락',
 '개미',
 '비명',
 '흉기',
 '휘두르',
 '정유정',
 '부친',
 '살인',
 '예고',
 '전화',
 '갑부',
 '고가',
 '익스트림',
 '여행',
 '사고',
 '수습',
 '비용',
 '부담',
 '우크라',
 '활약',
 '바그너

In [39]:
#중복되는 단어들의 개수를 세서 -> 표현이 최종 목표
df_word = pd.DataFrame({"word" : word})
#두 개이상의 단어만 선별
df_word["count"] = df_word["word"].str.len() #각 단어마다의 길이
df_word = df_word.query("count >=2 ") #query함수 사용해서 길이 2 이상인 단어만 뽑기
df_word

,word,count
1,주차장,3
3,차량,2
4,경찰,2
5,체포,2
6,압수,2
...,...,...
3100,장인,2
3101,정해인,3
3102,이번,2
3103,강렬하,3


In [40]:
#df_word에서 단어 중복 세기
df_word_sort = df_word.groupby("word", as_index=  False).count().sort_values("count", ascending= False) #내림차순ascending
df_word_sort #변수명 바꾸는 이유 : 덮어쓰기 진행됨.

,word,count
1127,정부,15
1129,정유정,14
636,새우깡,13
754,신라면,13
1021,인하,12
...,...,...
787,악셀,1
788,악취,1
789,안간힘,1
248,눈송이,1


In [41]:
df_word_sort = df_word_sort.query("count >= 2") #다시 개수가 2이상만 보기
df_word_sort

,word,count
1127,정부,15
1129,정유정,14
636,새우깡,13
754,신라면,13
1021,인하,12
...,...,...
236,노출,2
275,대결,2
172,기준,2
176,김건희,2
